# Analyze performance metrics of registration experiments

#### Author: 
Bruno De Santi, PhD
#### Affiliation:
Multi-modality Medical Imaging Lab (M3I Lab), University of Twente, Enschede, The Netherlands
#### Date:
20/09/2023
#### Paper/Project Title:
Automated three-dimensional image registration for longitudinal photoacoustic imaging (De Santi et al. 2023, JBO)
#### GitHub:
https://github.com/brunodesanti/muvinn-reg
#### License:
[Specify the license, e.g., MIT, GPL, etc.]

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 

# Font specifications
plt.rcParams.update({'font.size': 12})
plt.rcParams.update({'font.family': "sans-serif"})
plt.rcParams.update({'font.sans-serif': "Arial"})

### Load, analyze and plot performance metrics included in the specified folder

In [ ]:
# Name of the experiment
experiment_name = 'exp1'

# Path where metrics_before, metrics_after and execution_times files are stored
path = r'path\to\experiment\folder' 

# Output path where to save charts figure
output_path = r'path\to\output\folder'

# Load files and store values in the lists
metrics_before = []
metrics_after = []
execution_times = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and 'metrics_before' in i:
        metrics_before.append(np.load(os.path.join(path,i), allow_pickle=True).item())
    elif os.path.isfile(os.path.join(path,i)) and 'metrics_after' in i:
        metrics_after.append(np.load(os.path.join(path,i), allow_pickle=True).item())
    elif os.path.isfile(os.path.join(path,i)) and 'execution_time' in i:
        execution_times.append(np.load(os.path.join(path,i), allow_pickle=True).item())

# Convert from lists to numpy arrays for each metric
psnr_before = np.array([d['psnr'] for d in metrics_before])
ncc_before = np.array([d['ncc'] for d in metrics_before])
dsc_before = np.array([d['dice'] for d in metrics_before])
tre_before = np.array([d['tre'] for d in metrics_before])
tre_before = tre_before[:,0] # mean TRE

psnr_after = np.array([d['psnr'] for d in metrics_after])
ncc_after = np.array([d['ncc'] for d in metrics_after])
dsc_after = np.array([d['dice'] for d in metrics_after])
tre_after = np.array([d['tre'] for d in metrics_after])
tre_after = tre_after[:,0] # mean TRE

# Concatenate metrics in a single array
metrics_before = np.array([psnr_before, ncc_before, dsc_before, tre_before]).T
metrics_after = np.array([psnr_after, ncc_after, dsc_after, tre_after]).T

# Metric names
metrics_name = ('PSNR, dB','NCC','DSC','TRE, mm')

# Define y ranges
ymins = [30 - 0.1, -0.1, -0.1, -0.1] 
ymaxs = [50, 1, 1, 15]

# Select in order metrics to be plotted
ind_metrics = [0, 1, 2, 3]
fig, (ax) = plt.subplots(1, len(ind_metrics), figsize=(12, 3))

# Plot
for i, ind_metric in enumerate(ind_metrics):
    ax[i].scatter([-2]*len(metrics_before[:,ind_metric]),metrics_before[:,ind_metric],c='k')
    ax[i].scatter([2]*len(metrics_after[:,ind_metric]),metrics_after[:,ind_metric],c='k')
    ax[i].spines['top'].set_visible(False)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['bottom'].set_visible(False)
    ax[i].spines['left'].set_visible(False)
    ax[i].get_xaxis().set_ticks([])
    ax[i].set_xlim(xmin=-5,xmax=5)
    ax[i].set_ylim(ymin=ymins[ind_metric],ymax=ymaxs[ind_metric])
    for j in range(0,len(metrics_before[:,ind_metric])):
        ax[i].plot((-2,2),(metrics_before[:,ind_metric][j],metrics_after[:,ind_metric][j]),'--',c='k')
    ax[i].set_title(metrics_name[ind_metric])
    ax[i].grid()
plt.show()

# Save
fig.savefig(r'{}\metrics_charts_{}.png'.format(output_path, experiment_name))